This NB demonstrates different **SMOS Catalogs** that provide access to the SMOS Level-2 datasets in the archive.

The SMOS catalog can be passed to the **SMOS Store** constructor using the parameter `_catalog`. For example

```python
from xcube_smos.store import SmosDataStore
from xcube_smos.catalog import SmosIndexCatalog

store = SmosDataStore(_catalog=SmosIndexCatalog(index_path))
```

If used that way, all other SMOS store parameters no longer apply.
The default catalog is `SmosDirectCatalog`. 

In [1]:
import json
import os
from xcube_smos.catalog import SmosDirectCatalog
from xcube_smos.catalog import SmosIndexCatalog

**SmosDirectCatalog**: A catalog that accesses the NetCDF files in S3 directly.

In [2]:
with open("creodias-credentials.json") as f:
    credentials = json.load(f)

In [3]:
catalog = SmosDirectCatalog(
    source_path="s3://EODATA",
    source_storage_options={
        "endpoint_url": "https://s3.cloudferro.com",
        "anon": False,
        **credentials
    }
)

In [4]:
datasets = catalog.find_datasets("OS", ("2023-05-01", "2023-05-06"))

In [5]:
len(datasets)

66

In [6]:
dataset_path, start, end = datasets[0]

In [7]:
dataset_path

'EODATA/SMOS/L2OS/MIR_OSUDP2/2023/05/01/SM_OPER_MIR_OSUDP2_20230501T012809_20230501T022124_700_001_1/SM_OPER_MIR_OSUDP2_20230501T012809_20230501T022124_700_001_1.nc'

In [8]:
open_dataset = catalog.get_dataset_opener()

In [9]:
ds = open_dataset(    
    dataset_path,
    **catalog.get_dataset_opener_kwargs()
)
ds

<xarray.Dataset>
Dimensions:              (n_grid_points: 117970)
Dimensions without coordinates: n_grid_points
Data variables: (12/13)
    Dg_quality_SSS_corr  (n_grid_points) uint16 dask.array<chunksize=(117970,), meta=np.ndarray>
    Sigma_SSS_corr       (n_grid_points) float32 dask.array<chunksize=(117970,), meta=np.ndarray>
    Dg_RFI_X             (n_grid_points) uint16 dask.array<chunksize=(117970,), meta=np.ndarray>
    Dg_RFI_Y             (n_grid_points) uint16 dask.array<chunksize=(117970,), meta=np.ndarray>
    SSS_anom             (n_grid_points) float32 dask.array<chunksize=(117970,), meta=np.ndarray>
    Mean_acq_time        (n_grid_points) float32 dask.array<chunksize=(117970,), meta=np.ndarray>
    ...                   ...
    Dg_quality_SSS_anom  (n_grid_points) uint16 dask.array<chunksize=(117970,), meta=np.ndarray>
    X_swath              (n_grid_points) float32 dask.array<chunksize=(117970,), meta=np.ndarray>
    SSS_corr             (n_grid_points) float32 dask.array<chunksize=(117970,), meta=np.ndarray>
    Coast_distance       (n_grid_points) uint8 dask.array<chunksize=(117970,), meta=np.ndarray>
    Grid_Point_ID        (n_grid_points) uint32 dask.array<chunksize=(117970,), meta=np.ndarray>
    Sigma_SSS_anom       (n_grid_points) float32 dask.array<chunksize=(117970,), meta=np.ndarray>
Attributes:
    Precise_Validity_Start:  UTC=2023-05-01T01:28:08.370645
    Precise_Validity_Stop:   UTC=2023-05-01T02:21:24.015375
    Abs_Orbit_Start:         +70910
    Start_Time_ANX_T:        1320.166030
    Abs_Orbit_Stop:          +70910
    Stop_Time_ANX_T:         4515.810760
    UTC_at_ANX:              UTC=2023-05-01T01:06:08.204615
    Long_at_ANX:             +073.735974
    Ascending_Flag:          D
    Polarisation_Flag:       F

**SmosIndexCatalog**: A catalog that uses the SMOS Kerchunk Index.

In [10]:
index_path = os.environ.get("XCUBE_SMOS_INDEX_PATH")
catalog = SmosIndexCatalog(index_path)

In [11]:
datasets = catalog.find_datasets("OS", ("2023-05-01", "2023-05-06"))

In [12]:
len(datasets)

60

In [13]:
ref_json_path, start, end = datasets[0]

In [14]:
ref_json_url = catalog.resolve_path(ref_json_path)

In [15]:
ref_json_url

'zip://SMOS/L2OS/MIR_OSUDP2/2023/05/01/SM_OPER_MIR_OSUDP2_20230501T012809_20230501T022124_700_001_1/SM_OPER_MIR_OSUDP2_20230501T012809_20230501T022124_700_001_1.nc.json::file://C:\\Users\\norma\\nckc-index.zip'

In [16]:
open_dataset = catalog.get_dataset_opener()

In [17]:
ds = open_dataset(ref_json_url)

In [18]:
ds

<xarray.Dataset>
Dimensions:                  (n_grid_points: 117970)
Dimensions without coordinates: n_grid_points
Data variables: (12/64)
    A_card                   (n_grid_points) float32 ...
    Coast_distance           (n_grid_points) uint8 ...
    Control_Flags_Acard      (n_grid_points) uint32 ...
    Control_Flags_anom       (n_grid_points) uint32 ...
    Control_Flags_corr       (n_grid_points) uint32 ...
    Control_Flags_uncorr     (n_grid_points) uint32 ...
    ...                       ...
    Tb_42_5V                 (n_grid_points) float32 ...
    Tb_42_5X                 (n_grid_points) float32 ...
    Tb_42_5Y                 (n_grid_points) float32 ...
    WS                       (n_grid_points) float32 ...
    WS_corr                  (n_grid_points) uint16 ...
    X_swath                  (n_grid_points) float32 ...
Attributes: (12/2329)
    FH:File_Class:                                                           ...
    FH:File_Description:                                                     ...
    FH:File_Name:                                                            ...
    FH:File_Type:                                                            ...
    FH:File_Version:                                                         ...
    FH:Mission:                                                              ...
    ...                                                                                                   ...
    VH:SPH:QI:LORS:Quality_Description_3:SQ:WS_Class:                        ...
    VH:SPH:QI:LORS:Quality_Description_3:Too_Close_To_Land_Rejected:         ...
    VH:SPH:QI:LORS:Quality_Description_3:Too_Few_Measurements_Rejected:      ...
    VH:SPH:QI:Total_Selected_L1c_Grid_Points:                                ...
    creation_date:                                                           ...
    total_number_of_grid_points:                                             ...

In [19]:
ds.attrs.get("VH:SPH:MI:TI:Ascending_Flag")

'D'